In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [44]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
bs = 64
epochs = 10


Using cpu device


In [89]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 18),
            nn.ReLU(),
            nn.Linear(18, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
        
def get_model():
    model = NeuralNetwork()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    return model, loss_fn, optimizer

def get_data(training_data, test_data, bs):
    return (
        DataLoader(training_data, batch_size=bs), #, shuffle=True),
        DataLoader(test_data, batch_size=bs * 2),
    )

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    size = len(train_dl.dataset)
    for epoch in range(epochs):
        model.train()
        for batch, (xb, yb) in enumerate(train_dl):
            # xb = xb.to(device), yb.to(device)
            loss, _ = loss_batch(model, loss_func, xb, yb, opt)
            
            if batch % 100 == 0:
                current = (batch + 1) * len(xb)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        model.eval()
        with torch.no_grad():
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb, yb) for xb, yb in valid_dl]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)

        print(epoch, val_loss)


In [91]:
torch.manual_seed(0)
train_dl, valid_dl = get_data(training_data, test_data, bs)
model, loss_func, opt = get_model()
torch.save(model.state_dict(), "model.pth")
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

loss: 2.311446  [   64/60000]
loss: 2.318978  [ 6464/60000]
loss: 2.292953  [12864/60000]
loss: 2.208490  [19264/60000]
loss: 2.216908  [25664/60000]
loss: 2.212726  [32064/60000]
loss: 2.156218  [38464/60000]


KeyboardInterrupt: 

In [15]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()